In [3]:
from bs4 import BeautifulSoup
import lxml
import requests
import json
import re
import os
import regex
from tqdm import tqdm
import pickle
import socks
import socket

In [2]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 185.17.128.125


In [1]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'

In [5]:
response = requests.get("https://www.citywalls.ru/house1.html", headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [9]:
response = requests.get("https://www.citywalls.ru/house1.html")
for key, value in response.request.headers.items():
    print(key+": "+value)

User-Agent: python-requests/2.23.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive


In [4]:
#подключаемся к порту Tor
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [5]:
checkIP()

Current IP Address: 185.107.70.202


In [22]:
import time

In [23]:
#ip меняется каждые 10 секунд (надо повозится в настройках Tor)
for i in range(10):
    checkIP()
    time.sleep(5)

Current IP Address: 81.6.43.167
Current IP Address: 81.6.43.167
Current IP Address: 209.141.45.189
Current IP Address: 209.141.45.189
Current IP Address: 104.244.73.85
Current IP Address: 104.244.73.85
Current IP Address: 185.191.124.153
Current IP Address: 185.191.124.153
Current IP Address: 109.70.100.46
Current IP Address: 109.70.100.46


In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'}

In [ ]:
#32113 страниц со зданиями - скачаем их
for x in tqdm(range(32113)):
    response = requests.get(f"https://www.citywalls.ru/house{x}.html", headers=headers)
    if response.status_code == 200:
        page = unescape(response.text)
        page = normalize('NFKD', response.text)
        with open(f'citywalls/house{x}.html', 'w', encoding='1251', errors="ignore") as f:
            f.write(page)

1) берём содержимое div.address
2) делаем сплит по <br>
3) убираем html код (остаются только ссылки

In [ ]:
path = 'citywalls'
correct_styles = []
for i, file in tqdm(zip(range(33000), os.listdir(path=path))):
    if os.path.exists(f'{path}/{file}'):
        with open(f'{path}/{file}') as f:
            html = f.read()
            soup = BeautifulSoup(html, features='html.parser')
            if soup.find('div', class_='cssHouseHead'):
                if soup.find('<br/>'):
                    full_address = str(soup.find('div', class_ ='address')).split('<br/>')
                    split_adress = [re.sub('<[^>]*>', '', address) for address in full_address]
                metainfo = ('\n'.join([tag.text for tag in soup.find_all('div', {'class': 'cssHouseHead'})])).split('\n\n\n')
                if len(metainfo[-2]) < 10:
                    correct_styles.append({'Здание': metainfo[0].strip('\n'), 'Адрес': split_adress, 
                                            'Стиль': 'нет информации', 'Ссылка': f'https://www.citywalls.ru/{file}'})
                else:
                    correct_styles.append({'Здание': metainfo[0].strip('\n'), 'Адрес': split_adress, 
                'Стиль': metainfo[-2].strip('\n').split('\n')[1], 'Ссылка': f'https://www.citywalls.ru/{file}'})

In [7]:
correct_styles[:10]

[{'Здание': 'Училище правоведения',
  'Адрес': ['Чайковского ул., 1',
   'Фонтанки наб., 6',
   'Оружейника Федорова ул., 2'],
  'Стиль': 'Классицизм',
  'Ссылка': 'https://www.citywalls.ru/house1.html'},
 {'Здание': 'Дома Олсуфьевых (левая часть)',
  'Адрес': ['Фонтанки наб., 14'],
  'Стиль': 'Необарокко',
  'Ссылка': 'https://www.citywalls.ru/house10.html'},
 {'Здание': 'Евангелическо- лютеранская церковь св. Екатерины',
  'Адрес': ['Большой пр. ВО, 1'],
  'Стиль': 'Классицизм',
  'Ссылка': 'https://www.citywalls.ru/house100.html'},
 {'Здание': 'Училище св. Елены. Еленинское училище - Женскии институт св. ЕленыШкола No 77 (по ул. Блохина)',
  'Адрес': ['Блохина ул., 31', 'Зверинская ул., 3'],
  'Стиль': 'Эклектика',
  'Ссылка': 'https://www.citywalls.ru/house1000.html'},
 {'Здание': 'Детская музыкальная школа No 26 - Охтинскии центр эстетического воспитания',
  'Адрес': ['Маршала Тухачевского ул., 8'],
  'Стиль': 'Современныи',
  'Ссылка': 'https://www.citywalls.ru/house10000.html'},

In [89]:
import json
with open('citwalls_styles_with_links.json', 'w', encoding='utf-8') as f:
    json.dump(correct_styles, f, ensure_ascii=False, indent=4)